In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns

import sweetviz as sv

## Loading data

In [4]:
file_path = './analytics_test_ms_1.xlsx'

df = pd.read_excel(file_path, sheet_name='total_merged', index_col=0)

df.head()

,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date,has_tuning,has_airbags,...,gear_type,ac_type,xenon_light,navigation_system,radio_system,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status
car_id_sales,,,,,,,,,,,,,,,,,,,,,
3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31,1,1,...,1138,1050,0,0,3534.0,NaN,0,0,NaN,NaN
5096178328,DE,7833165.0,2021-07-13,2021-28,2021-07-19,15490,0,NaT,1,1,...,1141,1049,0,1100,3534.0,C1AA16434,0,0,NaN,CREATED
5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31,0,1,...,1138,0,0,0,NaN,NaN,0,0,NaN,NaN
7525691980,DE,NaN,2021-06-15,2021-24,2021-06-21,2721,1,2021-06-28,1,1,...,1139,1050,0,0,NaN,NaN,0,0,NaN,NaN
5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaT,0,1,...,1138,1050,0,1100,3534.0,NaN,0,0,NaN,NaN


## Dataset

In [5]:
df_total = df.copy()

In [6]:
df_total.head()

,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date,has_tuning,has_airbags,...,gear_type,ac_type,xenon_light,navigation_system,radio_system,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status
car_id_sales,,,,,,,,,,,,,,,,,,,,,
3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31,1,1,...,1138,1050,0,0,3534.0,NaN,0,0,NaN,NaN
5096178328,DE,7833165.0,2021-07-13,2021-28,2021-07-19,15490,0,NaT,1,1,...,1141,1049,0,1100,3534.0,C1AA16434,0,0,NaN,CREATED
5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31,0,1,...,1138,0,0,0,NaN,NaN,0,0,NaN,NaN
7525691980,DE,NaN,2021-06-15,2021-24,2021-06-21,2721,1,2021-06-28,1,1,...,1139,1050,0,0,NaN,NaN,0,0,NaN,NaN
5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaT,0,1,...,1138,1050,0,1100,3534.0,NaN,0,0,NaN,NaN


## Definition of target column - claim "yes" (1 / 0 ) column

### Recognition of claims that had complaint processing to the end and were not fully rejected. Those considered as current plaints were excluded from the dataset.

In [8]:
df_total['target_claim'] = df_total.apply(lambda row: 1 if (row['full_refunds'] == 1 or row['partial_refunds'] == 1) and row['claim_status'] != 'REJECTED' else 0, axis=1)


In [9]:
condition = (df_total['claim_number'].notnull()) & (df_total['full_refunds'] == 0) & (df_total['partial_refunds'] == 0)


In [10]:
rows_to_drop = df_total.loc[condition].index

# Usuń wybrane wiersze
df_total.drop(rows_to_drop, inplace=True)

In [11]:
df_total['target_claim'].value_counts()

target_claim
0    55152
1    11237
Name: count, dtype: int64

## Variables preprocessing - transforming some variables and adjusting data type

In [12]:
df_total['days_between_payment_and_selling'] = (df_total['payment_date'] - df_total['selling_date']).dt.days

In [13]:
df_total['days_between_payment_and_selling'].value_counts()

days_between_payment_and_selling
1      13218
2      11451
3       9090
4       8070
5       5219
       ...  
162        1
168        1
54         1
57         1
130        1
Name: count, Length: 103, dtype: int64

In [14]:
x = df_total[['days_between_payment_and_selling']]

In [15]:
kmeans = KMeans(n_clusters=6, random_state=42)
df_total['d_payment_and_selling_clustered'] = kmeans.fit_predict(x)

c:\Users\marsr\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
cluster_3_index = df_total[df_total['d_payment_and_selling_clustered'] == 3].index
cluster_5_index = df_total[df_total['d_payment_and_selling_clustered'] == 5].index

df_total.loc[cluster_3_index, 'd_payment_and_selling_clustered'] = 4
df_total.loc[cluster_5_index, 'd_payment_and_selling_clustered'] = 4

In [17]:
df_total['d_payment_and_selling_clustered'].value_counts()

d_payment_and_selling_clustered
1    36347
2    22819
0     6350
4      873
Name: count, dtype: int64

In [18]:
df_total['selling_week_number'] = df_total['selling_week'].str.split('-').str[1].astype(pd.Int32Dtype())

In [19]:
df_total['radio_system'].fillna(0, inplace=True)

In [20]:
df_total['radio_system'].astype(pd.Int32Dtype())

car_id_sales
3421775349    3534
5316743932       0
7525691980       0
5785159431    3534
1879501444    3534
              ... 
9115994556    3534
2548116105    1100
9754393407    3534
1548066157    3534
4594928165    3534
Name: radio_system, Length: 66389, dtype: Int32

In [21]:
columns_to_drop = ['merchant_id', 'selling_date', 'selling_week', 'payment_date', 
                   'transport_date', 'claim_number', 'full_refunds', 
                   'partial_refunds', 'voucher_amount', 'claim_status', 'days_between_payment_and_selling']

In [22]:
df_total.drop(columns=columns_to_drop, inplace=True)

In [23]:
df_total['country'] = df_total['country'].astype('category')
df_total['radio_system'] = df_total['radio_system'].astype('int')

In [24]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66389 entries, 3421775349 to 4594928165
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   country                          66389 non-null  category
 1   sell_price                       66389 non-null  int64   
 2   transport_to_merchant            66389 non-null  int64   
 3   has_tuning                       66389 non-null  int64   
 4   has_airbags                      66389 non-null  int64   
 5   has_alarm_system                 66389 non-null  int64   
 6   fuel_type                        66389 non-null  int64   
 7   gear_type                        66389 non-null  int64   
 8   ac_type                          66389 non-null  int64   
 9   xenon_light                      66389 non-null  int64   
 10  navigation_system                66389 non-null  int64   
 11  radio_system                     66389 non-null  int32   


In [25]:
df_total.describe()

,sell_price,transport_to_merchant,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system,target_claim,d_payment_and_selling_clustered,selling_week_number
count,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.000000,66389.0
mean,7332.026043,0.508217,0.369579,0.847806,0.159108,1011.640347,1138.874904,850.400428,171.742759,385.560861,1799.790116,0.169260,1.287518,21.633433
std,8399.883190,0.499936,0.482694,0.359211,0.365779,168.159197,1.243893,411.630982,396.906364,524.846567,1641.274055,0.374984,0.689241,9.09402
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1138.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
25%,2215.000000,0.000000,0.000000,1.000000,0.000000,1039.000000,1138.000000,1049.000000,0.000000,0.000000,0.000000,0.000000,1.000000,14.0
50%,4749.000000,1.000000,0.000000,1.000000,0.000000,1040.000000,1138.000000,1050.000000,0.000000,0.000000,1100.000000,0.000000,1.000000,22.0
75%,9317.000000,1.000000,1.000000,1.000000,0.000000,1040.000000,1139.000000,1050.000000,0.000000,1100.000000,3534.000000,0.000000,2.000000,29.0
max,173000.000000,1.000000,1.000000,1.000000,1.000000,1040.000000,1141.000000,1050.000000,1089.000000,1100.000000,3534.000000,1.000000,4.000000,38.0


## Visualizations of pairs: variables and target_claim (using sweetviz library)


In [26]:
import sweetviz as sv

report = sv.analyze(df_total, target_feat='target_claim')

report.show_html('sweetviz_report.html')

                                             |          | [  0%]   00:00 -> (? left)

AttributeError: 'float' object has no attribute 'shape'